In [184]:
l1_layers = [1, 2, 3]
layer_sizes = [16, 32, 64]
l2_layers = [1, 2, 3]

for l1_layer in l1_layers:
    for layer_size in layer_sizes:
        for l2_layer in l2_layers:
            NAME = "{}-l1-{}-l2-{}-nodes-{}".format(l1_layer, l2_layer, layer_size,int(time.time()))
            print(NAME)

            model = Sequential()
            model.add(Dense(output_dim = layer_size, init = 'uniform', activation = 'relu', input_dim = 9))
            for _ in range(l1_layer):
                model.add(Dense(output_dim = layer_size, init = 'uniform', activation = 'relu',kernel_regularizer=regularizers.l1(0.01)))

            for l in range(l2_layer):
                model.add(Dense(output_dim = layer_size, init = 'uniform', activation = 'relu', kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.fit(X_train,np.array(y_train),
                      batch_size=10,
                      epochs=30,
                      validation_split=0.2, 
                      callbacks = [tensorboard])

1-l1-1-l2-16-nodes-1535442972


/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=9, activation="relu", units=16, kernel_initializer="uniform")`
  if sys.path[0] == '':
/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=16, kernel_initializer="uniform", kernel_regularizer=<keras.reg...)`
  
/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=16, kernel_initializer="uniform", kernel_regularizer=<keras.reg...)`
/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="sigmoi

Train on 12540 samples, validate on 3136 samples
Epoch 1/30


FailedPreconditionError: Attempting to use uninitialized value training_21/Adam/Variable_4
	 [[Node: training_21/Adam/Variable_4/read = Identity[T=DT_FLOAT, _class=["loc:@training_21/Adam/Assign_12"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](training_21/Adam/Variable_4)]]

In [17]:
from keras.models import model_from_json
json_file = open('modelv5.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelv5.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Loaded model from disk


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
import time
import tensorflow as tf
import imblearn.over_sampling

/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
dataset = pd.read_csv('csv2.csv')

In [20]:
dataset = pd.read_csv('csv2.csv', error_bad_lines=False)
dataset= dataset.dropna(how = 'any')
X = dataset[['starting frequency (kHz)','final frequency (kHz)','minimum frequency (kHz)','maximum frequency (kHz)','mean frequency (kHz)','frequency bandwidth (kHz)','syllable duration (msec)','total syllable energy (dB)','peak syllable amplitude (dB)']]
Y = dataset.iloc[:,12]


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
sm = imblearn.over_sampling.SMOTE(random_state= 12, ratio= 1.0)
X_trainr, y_trainr = sm.fit_sample(X_train, y_train)


/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
Xc =sc.fit_transform(X)

In [147]:
#Initializing Neural Network
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 30, init = 'uniform', activation = 'relu', input_dim = 9), )

classifier.add(Dense(output_dim = 30, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
# Compiling Neural Network"
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Fitting our model 
moniter = EarlyStopping(monitor = 'acc', min_delta = 1e-03, patience = 5, verbose = 1, mode = 'auto')
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, callbacks = [moniter])

/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=9, activation="relu", units=30, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", units=30)`
  
/Users/sebastianmensink/miniconda3/envs/python/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="sigmoid", units=1)`
  import sys


Epoch 1/100


KeyboardInterrupt: 

In [23]:
# Predicting the Test set results
y_pred = loaded_model.predict(X_test)
y_pred = (y_pred > 0.5)

In [24]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 317   71]
 [  95 3442]]
             precision    recall  f1-score   support

          0       0.77      0.82      0.79       388
          1       0.98      0.97      0.98      3537

avg / total       0.96      0.96      0.96      3925



In [21]:
np.array(y_train)

array([1., 1., 1., ..., 1., 1., 1.])

In [56]:
# serialize model to JSON
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [170]:
Xc

array([[-1.19912859, -0.57638322, -1.65912019, ..., -0.47704444,
        -1.19203023, -1.29247631],
       [-1.71131759, -0.87037139, -1.35780859, ..., -0.67129015,
        -0.91534617, -1.22592196],
       [-0.05905136, -2.0282839 , -1.65912019, ...,  6.64529794,
         0.01341072, -0.35249888],
       ...,
       [ 0.05664787,  1.52763668,  0.14936688, ...,  0.36468693,
         0.41056115,  0.34673259],
       [ 0.13919352, -0.59308709, -0.10687147, ...,  0.10569266,
         0.33284091,  0.45847816],
       [ 0.48629122, -0.44609301,  0.05922037, ..., -0.54179301,
         0.13076828,  0.22677043]])

In [180]:
x = np.array([0,1,0,1,1,0])
x[x<1] = -1
print (x)

[-1  1 -1  1  1 -1]
